## Part 3: Machine Learning
#### 3.1 Data Preparation
The taxi database undergoes basic machine learning relevant preprocessing steps.

In [0]:
spark.sql("DROP TABLE IF EXISTS taxi_database.taxi_table")
spark.sql("DROP DATABASE IF EXISTS taxi_database")
spark.sql("CREATE DATABASE taxi_database")
spark.sql("USE taxi_database")
spark.sql("CREATE TABLE taxi_table USING PARQUET OPTIONS (path '/dbfs/mnt_1/taxi_combined.parquet')")

#### 3.1.1 Drop October to December 2022

In [0]:
taxi_ml = spark.sql("""
                   SELECT *
                   FROM taxi_table
                   WHERE lpep_pickup_datetime < '2022-10-01T00:00:00.000'
                   """)

#### 3.1.2 Dropping Fare Amount

In [0]:
taxi_ml = taxi_ml.drop('fare_amount')

#### 3.1.3 Creating Features useful for Machine Learning

Since the timestamp column cannot be used, a column representing the year and month, day of the week, and the hour of the day are created instead. To provide the model with maximum context, the pickup and dropoff zones are added as well from the reference table.

In [0]:
# Adding time related details onto the dataset
from pyspark.sql.functions import date_format, col, hour

taxi_ml = taxi_ml.withColumn('year_month', date_format(col('lpep_pickup_datetime'), 'yyyy-MM'))
taxi_ml = taxi_ml.withColumn('day_of_week', date_format(col('lpep_pickup_datetime'), 'E'))
taxi_ml = taxi_ml.withColumn('hour_of_day', hour(col('lpep_pickup_datetime')))

In [0]:
# Adding pcikup and dropoff zone information
reference = spark.read.csv("/FileStore/taxi_zone_lookup.csv", header=True, inferSchema=True)
reference.createOrReplaceTempView('ref')
taxi_ml.createOrReplaceTempView('taxi_table')

taxi_ml = spark.sql("""
                    SELECT t.*,
                        pickup.Zone AS pickup_zone,
                        dropoff.Zone AS dropoff_zone
                      FROM taxi_table AS t
                      LEFT JOIN ref AS pickup
                        ON t.PULocationID = pickup.LocationID
                      LEFT JOIN ref as dropoff
                        ON t.DOLocationID = dropoff.LocationID
                    """)

In [0]:
taxi_ml.display()

lpep_pickup_datetime lpep_dropoff_datetime PULocationID DOLocationID passenger_count extra tip_amount total_amount trip_duration_hours trip_distance_km speed_kmph taxi_color mta_and_improvement pickup_borough dropoff_borough year_month day_of_week hour_of_day pickup_zone dropoff_zone 2015-01-01T00:45:25Z 2015-01-01T01:02:09Z 7 168 1.0 0.5 0.0 22.8 0.28 10.94 39.07 yellow 0.8 Queens Bronx 2015-01 Thu 0 Astoria Mott Haven/Port Morris 2015-01-01T00:18:42Z 2015-01-01T01:10:58Z 181 142 1.0 0.5 0.0 38.8 0.87 14.92 17.15 yellow 0.8 Brooklyn Manhattan 2015-01 Thu 0 Park Slope Lincoln Square East 2015-01-01T00:43:46Z 2015-01-01T01:09:26Z 262 47 5.0 0.5 0.0 29.3 0.43 13.36 31.07 yellow 0.8 Manhattan Bronx 2015-01 Thu 0 Yorkville East Claremont/Bathgate 2015-01-01T00:46:17Z 2015-01-01T01:12:06Z 151 79 1.0 0.5 5.1 31.4 0.43 10.94 25.44 yellow 0.8 Manhattan Manhattan 2015-01 Thu 0 Manhattan Valley East Village 2015-01-01T00:23:15Z 2015-01-01T00:39:57Z 164 229 2.0 0.5 3.0 16.3 0.28 2.9 10.36 yellow 0.8 Manhattan Manhattan 2015-01 Thu 0 Midtown South Sutton Place/Turtle Bay North 2015-01-01T00:14:40Z 2015-01-01T00:25:54Z 262 24 1.0 0.5 0.0 12.8 0.19 4.7 24.74 yellow 0.8 Manhattan Manhattan 2015-01 Thu 0 Yorkville East Bloomingdale 2015-01-01T00:39:19Z 2015-01-01T01:09:27Z 163 158 3.0 0.5 0.0 20.8 0.5 5.2 10.4 yellow 0.8 Manhattan Manhattan 2015-01 Thu 0 Midtown North Meatpacking/West Village West 2015-01-01T00:20:05Z 2015-01-01T00:30:06Z 114 79 4.0 0.5 2.7 12.5 0.17 2.11 12.41 yellow 0.8 Manhattan Manhattan 2015-01 Thu 0 Greenwich Village South East Village 2015-01-01T00:56:18Z 2015-01-01T01:05:16Z 137 229 2.0 0.5 0.0 8.8 0.15 2.17 14.47 yellow 0.8 Manhattan Manhattan 2015-01 Thu 0 Kips Bay Sutton Place/Turtle Bay North 2015-01-01T00:46:09Z 2015-01-01T01:06:07Z 144 162 2.0 0.5 3.15 18.95 0.33 4.99 15.12 yellow 0.8 Manhattan Manhattan 2015-01 Thu 0 Little Italy/NoLiTa Midtown East 2015-01-01T00:41:33Z 2015-01-01T00:49:11Z 80 255 1.0 0.5 1.5 9.8 0.13 2.22 17.08 yellow 0.8 Brooklyn Brooklyn 2015-01 Thu 0 East Williamsburg Williamsburg (North Side) 2015-01-01T00:25:54Z 2015-01-01T00:42:47Z 228 228 1.0 0.5 0.0 18.8 0.28 8.18 29.21 yellow 0.8 Brooklyn Brooklyn 2015-01 Thu 0 Sunset Park West Sunset Park West 2015-01-01T00:05:44Z 2015-01-01T00:16:56Z 237 7 2.0 0.5 0.0 14.8 0.19 6.29 33.11 yellow 0.8 Manhattan Queens 2015-01 Thu 0 Upper East Side South Astoria 2015-01-01T00:39:25Z 2015-01-01T01:18:09Z 246 36 1.0 0.5 3.88 33.88 0.65 11.75 18.08 yellow 0.8 Manhattan Brooklyn 2015-01 Thu 0 West Chelsea/Hudson Yards Bushwick North 2015-01-01T00:57:12Z 2015-01-01T01:04:06Z 170 229 3.0 0.5 1.5 9.8 0.12 2.27 18.92 yellow 0.8 Manhattan Manhattan 2015-01 Thu 0 Murray Hill Sutton Place/Turtle Bay North 2015-01-01T00:50:23Z 2015-01-01T01:02:21Z 263 162 1.0 0.5 2.88 13.88 0.2 3.22 16.1 yellow 0.8 Manhattan Manhattan 2015-01 Thu 0 Yorkville West Midtown East 2015-01-01T00:15:05Z 2015-01-01T00:29:01Z 50 236 1.0 0.5 2.8 16.8 0.23 4.99 21.7 yellow 0.8 Manhattan Manhattan 2015-01 Thu 0 Clinton West Upper East Side North 2015-01-01T00:28:09Z 2015-01-01T00:43:39Z 170 68 1.0 0.5 0.0 12.8 0.26 3.38 13.0 yellow 0.8 Manhattan Manhattan 2015-01 Thu 0 Murray Hill East Chelsea 2015-01-01T00:59:58Z 2015-01-01T01:21:08Z 181 137 1.0 0.5 4.6 28.4 0.35 11.09 31.69 yellow 0.8 Brooklyn Manhattan 2015-01 Thu 0 Park Slope Kips Bay 2015-01-01T00:08:01Z 2015-01-01T00:23:23Z 25 107 3.0 0.5 0.0 16.3 0.26 6.28 24.15 yellow 0.8 Brooklyn Manhattan 2015-01 Thu 0 Boerum Hill Gramercy 2015-01-01T00:34:15Z 2015-01-01T00:59:53Z 249 255 1.0 0.5 0.0 20.8 0.43 7.24 16.84 yellow 0.8 Manhattan Brooklyn 2015-01 Thu 0 West Village Williamsburg (North Side) 2015-01-01T00:41:24Z 2015-01-01T00:54:12Z 166 142 1.0 0.5 3.05 15.35 0.21 3.38 16.1 yellow 0.8 Manhattan Manhattan 2015-01 Thu 0 Morningside Heights Lincoln Square East 2015-01-01T00:49:42Z 2015-01-01T01:02:50Z 246 164 1.0 0.5 2.15 12.95 0.22 2.09 9.5 yellow 0.8 Manhattan Manhattan 2015-01 Thu 0 West Chelsea/Hudson Yards Midtown South 2015-01-01T00:16

In [0]:
from pyspark.sql import functions as F

taxi_ml_sampled = taxi_ml.orderBy(F.rand()).limit(100000)

#### 3.1.5 Encode the string columns
SringIndexer is used to encode the non integer columns

In [0]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer

# List of columns to be indexed
columns_to_index = {
    "taxi_color": "taxi_color_encoded",
    "year_month": "year_month_encoded",
    "day_of_week": "day_of_week_encoded",
    "pickup_borough": "pickup_borough_encoded",
    "dropoff_borough": "dropoff_borough_encoded",
    "pickup_zone": "pickup_zone_encoded",
    "dropoff_zone": "dropoff_zone_encoded"
}

# Create a list of StringIndexer stages
indexers = [StringIndexer(inputCol=input_col, outputCol=output_col) for input_col, output_col in columns_to_index.items()]

# Create the pipeline with the list of indexers
pipeline = Pipeline(stages=indexers)

# Fit the pipeline and transform the data
taxi_ml_sampled = pipeline.fit(taxi_ml_sampled).transform(taxi_ml_sampled)

# Drop the original columns
taxi_ml_sampled = taxi_ml_sampled.drop(*columns_to_index.keys())

# Rename encoded columns back to original names
for input_col, output_col in columns_to_index.items():
    taxi_ml_sampled = taxi_ml_sampled.withColumnRenamed(output_col, input_col)

In [0]:
# Cache the result to optimize further operations
taxi_ml_sampled.cache()

# Save the resulting dataframe
taxi_ml_sampled.write.format("parquet").save("/dbfs/mnt_1/taxi_ml_sampled")

### 3.2 Baseline Model

Before condicting machine learning, a baseline model has been defined as a comparitive parameter. For the baseline model, a simple average of the total amount paid has been considered for each pair of zonewise dropoffs. Using this, the RSME score is calculated against the actual amount paid.

In [0]:
# Calculate and add the average amount paid column
taxi_ml_sampled.createOrReplaceTempView('taxi_table')
average = spark.sql("""
                    SELECT AVG(total_amount) AS average_amount,
                      pickup_borough,
                      dropoff_borough
                    FROM taxi_table
                    GROUP BY 
                      pickup_borough,
                      dropoff_borough
                    """)

In [0]:
# Join the average baseline to the main taxi dataframe
average.createOrReplaceTempView('average')
taxi_ml_sampled.createOrReplaceTempView('taxi_sampled')

# Ensure proper join between the datasets
taxi_ml_baseline = spark.sql("""
                    SELECT t.*,
                           COALESCE(a.average_amount, 0) AS average_amount -- Handle nulls
                    FROM taxi_sampled AS t
                    LEFT JOIN average AS a
                           ON t.pickup_borough = a.pickup_borough AND t.dropoff_borough = a.dropoff_borough
                    """)

#### 3.2.3 Calculate the Baseline RMSE

In [0]:
# Calculate the RMSE score
from pyspark.ml.evaluation import RegressionEvaluator

# Ensure 'average_amount' is used as the prediction column, and 'total_amount' is the actual label
evaluator = RegressionEvaluator(labelCol="total_amount", predictionCol="average_amount", metricName="rmse")

# Evaluate the baseline RMSE on the validation set
baseline_rmse = evaluator.evaluate(taxi_ml_baseline)

# Print the Baseline Model RMSE
print(f"Baseline Model RMSE: {baseline_rmse}")

Baseline Model RMSE: 8.80021234031032
